## Quickstart

### Introduction

Hi! Glad to see you made it to the Quickstart guide! In this guide you are introduced to how CARTOframes can be used by data scientists in spatial analysis workflows. Using simulated Starbucks revenue data, this guide walks through some common steps a data scientist takes to answer the following question: which stores are performing better than others?

Before you get started, we encourage you to have CARTOframes installed so you can get a feel for the library by using it:

In [1]:
# ! pip install cartoframes

For additional ways to install CARTOframes, check out the [Installation Guide](/developers/cartoframes/guides/Installation).

#### Spatial analysis scenario

Let's say you are a data scientist working for Starbucks and you want to better understand why some stores in Brooklyn, New York, perform better than others.

To begin, let's outline a workflow:

- Get and explore your company's data
- Create areas of influence for your stores
- Enrich your data with demographic data
- And finally, share the results of your analysis with your team

Let's get started!

### Get and explore your company's data

[Use this dataset](http://libs.cartocdn.com/cartoframes/files/starbucks_brooklyn.csv) to start your exploration. It contains information about the location of Starbucks and each store's annual revenue.

As a first exploratory step, read it into a Jupyter Notebook using [pandas](https://pandas.pydata.org/).

In [2]:
import pandas as pd

stores_df = pd.read_csv('http://libs.cartocdn.com/cartoframes/files/starbucks_brooklyn.csv')
stores_df.head()

,name,address,revenue
0,Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772
1,607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418
2,65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699
3,Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676
4,Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411


To display your stores as points on a map, you first have to convert the `address` column into geometries. This process is called [geocoding](https://carto.com/help/working-with-data/geocoding-types/) and CARTO provides a straightforward way to do it (you can learn more about it in the [Location Data Services guide](/developers/cartoframes/guides/Location-Data-Services)).

In order to geocode, you have to set your CARTO credentials. If you aren't sure about your API key, check the [Authentication guide](/developers/cartoframes/guides/Authentication/) to learn how to get it. In case you want to see the geocoded result, without being logged in, [you can get it here](http://libs.cartocdn.com/cartoframes/files/starbucks_brooklyn_geocoded.csv).

> Note: If you don't have an account yet, you can get a trial, or a free account if you are a student, by [signing up here](https://carto.com/signup/).

In [4]:
from cartoframes.auth import set_default_credentials

set_default_credentials('creds.json')

Now that your credentials are set, we are ready to geocode the dataframe. The resulting data will be a [GeoDataFrame](http://geopandas.org/data_structures.html#geodataframe).

In [5]:
from cartoframes.data.services import Geocoding

stores_gdf, _ = Geocoding().geocode(stores_df, street='address')
stores_gdf.head()

Success! Data geocoded correctly


,the_geom,name,address,revenue,gc_status_rel,carto_geocode_hash
0,POINT (-73.96561 40.67241),Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772,0.91,9212e0e908d8c64d07c6a94827322397
1,POINT (-73.96122 40.57796),607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418,0.97,b1bbfe2893914a350193969a682dc1f5
2,POINT (-73.98981 40.61950),65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699,0.94,e47cf7b16d6c9b53c63e86a0418add1d
3,POINT (-74.02767 40.63204),Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676,0.94,2f21749c02f73116892eb3b6fd5d5738
4,POINT (-74.00098 40.59321),Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411,0.95,134c23973313802448365db6235783f9


Done! Now that the stores are geocoded, you will notice a new column named `geometry` has been added. This column stores the geographic location of each store and it's used to plot each location on the map.

You can quickly visualize your geocoded dataframe using the Map and Layer classes. Check out the [Visualization guide](/developers/cartoframes/guides/Visualization) to learn more about the visualization capabilities inside of CARTOframes.

In [6]:
from cartoframes.viz import Map, Layer

Map(Layer(stores_gdf))

Great! You have a map!

With the stores plotted on the map, you now have a better sense about where each one is. To continue your exploration, you want to know which stores earn the most yearly revenue. To do this, you can use the [`size_continuous_style`](/developers/cartoframes/examples/#example-size-continuous-style) visualization layer:

In [8]:
from cartoframes.viz import Map, Layer, size_continuous_style

Map(Layer(stores_gdf, size_continuous_style('revenue'), title='Annual Revenue ($)'))

Good job! By using the [`size continuous visualization style`](/developers/cartoframes/examples/#example-size-continuous-style) you can see right away where the stores with higher revenue are. By default, visualization styles also provide a popup with the mapped value and an appropriate legend.

### Create your areas of influence

Similar to geocoding, there is a straightforward method for creating isochrones to define areas of influence around each store. Isochrones are concentric polygons that display equally calculated levels over a given surface area measured by time.

For this analysis, let's create isochrones for each store that cover the area within a 15 minute walk.

To do this you will use the Isolines data service:


In [9]:
from cartoframes.data.services import Isolines

isochrones_gdf, _ = Isolines().isochrones(stores_gdf, [15*60], mode='walk')
isochrones_gdf.head()

Success! Isolines created correctly


,source_id,data_range,lower_data_range,the_geom,range_label
0,0,900,0,"MULTIPOLYGON (((-73.96575 40.68146, -73.96762 ...",15 min.
1,1,900,0,"MULTIPOLYGON (((-73.96187 40.58632, -73.96288 ...",15 min.
2,2,900,0,"MULTIPOLYGON (((-73.99025 40.62750, -73.99155 ...",15 min.
3,3,900,0,"MULTIPOLYGON (((-74.02859 40.64105, -74.02931 ...",15 min.
4,4,900,0,"MULTIPOLYGON (((-74.00110 40.60186, -74.00249 ...",15 min.


In [10]:
stores_map = Map([
    Layer(isochrones_gdf),
    Layer(stores_gdf)
])

stores_map

There they are! To learn more about creating isochrones and isodistances check out the [Location Data Services guide](/developers/cartoframes/guides/Location-Data-Services).

> Note: You will see how to publish a map in the last section. If you already want to publish this map, you can do it by calling `stores_map.publish('starbucks_isochrones', password=None)`.

### Enrich your data with demographic data

Now that you have the area of influence calculated for each store, let's take a look at how to augment the result with population information to help better understand a store's average revenue per person.

> Note: To be able to use the Enrichment functions you need an enterprise CARTO account with Data Observatory 2.0 enabled. Contact your CSM or contact us at [sales@carto.com](mailto:sales@carto.com) for more information.


First, let's find the demographic variable we need. We will use the `Catalog` class that can be filter by country and category. In our case, we have to look for USA demographics datasets. Let's see which public ones are available.

In [11]:
from cartoframes.data.observatory import Catalog

datasets_df = Catalog().country('usa').category('demographics').datasets.to_dataframe()
datasets_df[datasets_df['is_public_data'] == True]

,id,slug,name,description,country_id,geography_id,geography_name,geography_description,category_id,category_name,provider_id,provider_name,data_source_id,lang,temporal_aggregation,time_coverage,update_frequency,version,is_public_data
0,carto-do-public-data.usa_acs.demographics_soci...,acs_sociodemogr_b758e778,5-yr ACS data at Census Block Groups level (20...,The American Community Survey (ACS) is an ongo...,usa,carto-do-public-data.usa_carto.geography_usa_b...,Census Block Groups (2015) - shoreline clipped,Shoreline clipped TIGER/Line boundaries. More ...,demographics,Demographics,usa_acs,USA American Community Survey,sociodemographics,eng,5yrs,"[2013-01-01,2018-01-01)",None,20132017,True
2,carto-do-public-data.usa_acs.demographics_soci...,acs_sociodemogr_97b40c3b,1-yr ACS data at States level (2009),The American Community Survey (ACS) is an ongo...,usa,carto-do-public-data.usa_carto.geography_usa_s...,States (2015) - shoreline clipped,Shoreline clipped TIGER/Line boundaries. More ...,demographics,Demographics,usa_acs,USA American Community Survey,sociodemographics,eng,yearly,"[2009-01-01,2010-01-01)",None,2009,True
3,carto-do-public-data.usa_acs.demographics_soci...,acs_sociodemogr_8074b548,1-yr ACS data at States level (2011),The American Community Survey (ACS) is an ongo...,usa,carto-do-public-data.usa_carto.geography_usa_s...,States (2015) - shoreline clipped,Shoreline clipped TIGER/Line boundaries. More ...,demographics,Demographics,usa_acs,USA American Community Survey,sociodemographics,eng,yearly,"[2011-01-01,2012-01-01)",None,2011,True
4,carto-do-public-data.usa_acs.demographics_soci...,acs_sociodemogr_f01e41c7,1-yr ACS data at States level (2014),The American Community Survey (ACS) is an ongo...,usa,carto-do-public-data.usa_carto.geography_usa_s...,States (2015) - shoreline clipped,Shoreline clipped TIGER/Line boundaries. More ...,demographics,Demographics,usa_acs,USA American Community Survey,sociodemographics,eng,yearly,"[2014-01-01,2015-01-01)",None,2014,True
5,carto-do-public-data.usa_acs.demographics_soci...,acs_sociodemogr_197de4f2,1-yr ACS data at States level (2012),The American Community Survey (ACS) is an ongo...,usa,carto-do-public-data.usa_carto.geography_usa_s...,States (2015) - shoreline clipped,Shoreline clipped TIGER/Line boundaries. More ...,demographics,Demographics,usa_acs,USA American Community Survey,sociodemographics,eng,yearly,"[2012-01-01,2013-01-01)",None,2012,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,carto-do-public-data.usa_acs.demographics_soci...,acs_sociodemogr_d5724bfb,5-yr ACS data at 5-digit Zip Code Tabluation A...,The American Community Survey (ACS) is an ongo...,usa,carto-do-public-data.usa_carto.geography_usa_z...,5-digit Zip Code Tabluation Areas (2015) - sho...,Shoreline clipped TIGER/Line boundaries. More ...,demographics,Demographics,usa_acs,USA American Community Survey,sociodemographics,eng,5yrs,"[2009-01-01,2014-01-01)",None,20092013,True
68,carto-do-public-data.usa_acs.demographics_soci...,acs_sociodemogr_8d5a6f8c,5-yr ACS data at 5-digit Zip Code Tabluation A...,The American Community Survey (ACS) is an ongo...,usa,carto-do-public-data.usa_carto.geography_usa_z...,5-digit Zip Code Tabluation Areas (2015) - sho...,Shoreline clipped TIGER/Line boundaries. More ...,demographics,Demographics,usa_acs,USA American Community Survey,sociodemographics,eng,5yrs,"[2010-01-01,2015-01-01)",None,20102014,True
69,carto-do-public-data.usa_acs.demographics_soci...,acs_sociodemogr_c73d76aa,5-yr ACS data at 5-digit Zip Code Tabluation A...,The American Community Survey (ACS) is an ongo...,usa,carto-do-public-data.usa_carto.geography_usa_z...,5-digit Zip Code Tabluation Areas (2015) - sho...,Shoreline clipped TIGER/Line boundaries. More ...,demographics,Demographics,usa_acs,USA American Community Survey,sociodemographics,eng,5yrs,"[2011-01-01,2016-01-01)",None,20112015,True
72,carto-do-public-data.usa_acs.demographics_soci...,acs_sociodemogr_19945

Nice! Let's take the first one (`acs_sociodemogr_b758e778`) that has aggregated data from 2013 to 2018 and check which of its variables have data about the total population.

In [12]:
from cartoframes.data.observatory import Dataset

dataset = Dataset.get('acs_sociodemogr_b758e778')
variables_df = dataset.variables.to_dataframe()
variables_df[variables_df['description'].str.contains('total population', case=False, na=False)]

,id,slug,name,description,column_name,db_type,dataset_id,agg_method,variable_group_id,starred
116,carto-do-public-data.usa_acs.demographics_soci...,income_per_capi_8a9352e0,income_per_capita,Per Capita Income in the past 12 Months. Per c...,income_per_capita,FLOAT,carto-do-public-data.usa_acs.demographics_soci...,AVG,None,None
131,carto-do-public-data.usa_acs.demographics_soci...,total_pop_3cf008b3,total_pop,Total Population. The total number of all peop...,total_pop,FLOAT,carto-do-public-data.usa_acs.demographics_soci...,SUM,None,None


We can see the variable that contains the total population is the one with the slug `total_pop_3cf008b3`. Now we are ready to enrich our areas of influence with that variable.

In [13]:
from cartoframes.data.observatory import Variable
from cartoframes.data.observatory import Enrichment

variable = Variable.get('total_pop_3cf008b3')

isochrones_gdf = Enrichment().enrich_polygons(isochrones_gdf, [variable])
isochrones_gdf.head()

,source_id,data_range,lower_data_range,the_geom,range_label,total_pop
0,0,900,0,"MULTIPOLYGON (((-73.96575 40.68146, -73.96762 ...",15 min.,60261.702456
1,1,900,0,"MULTIPOLYGON (((-73.96187 40.58632, -73.96288 ...",15 min.,47012.672405
2,2,900,0,"MULTIPOLYGON (((-73.99025 40.62750, -73.99155 ...",15 min.,64820.382546
3,3,900,0,"MULTIPOLYGON (((-74.02859 40.64105, -74.02931 ...",15 min.,51044.611294
4,4,900,0,"MULTIPOLYGON (((-74.00110 40.60186, -74.00249 ...",15 min.,20597.477964


Great! Let's see the result on a map:

In [15]:
from cartoframes.viz import color_continuous_style

Map(
  Layer(
    isochrones_gdf,
    color_continuous_style('total_pop'),
    title='Population'
  )
)

We can see that the area of influence of the store on the right, is the one with the highest population. Let's go a bit further and calculate and visualize the average revenue per person.

In [16]:
stores_gdf['rev_pop'] = stores_gdf['revenue']/isochrones_gdf['total_pop']

Map(Layer(stores_gdf, size_continuous_style('rev_pop'), title='Revenue per person ($)'))

As we can see, the store in the center is the one that has lower revenue per person. This insight will help us to focus on them in further analyses.

To learn more about discovering the data you want, check out the [data discovery guide](/developers/cartoframes/guides/Data-discovery). To learn more about enriching your data check out the [data enrichment guide](/developers/cartoframes/guides/Data-enrichment/).

### Publish and share your results

The final step in the workflow is to share this interactive map with your colleagues so they can explore the information on their own. Let's do it!

First, let's upload the data to CARTO to see how we can visualize CARTO tables and how it helps you to publish your maps. To upload your data, you just need to call `to_carto` with your GeoDataFrame:

In [17]:
from cartoframes import to_carto

to_carto(stores_gdf, 'starbucks_stores', if_exists='replace')
to_carto(isochrones_gdf, 'starbucks_isochrones', if_exists='replace')

Success! Data uploaded to table "starbucks_stores" correctly
Success! Data uploaded to table "starbucks_isochrones" correctly


Now, let's visualize them and add widgets to them so people are able to see some graphs of the information and filter it. To do this, we only have to add `default_widget=True` to the visualization layers.

In [19]:
result_map = Map([
    Layer(
      'starbucks_isochrones',
      color_continuous_style('total_pop', stroke_width=0, opacity=0.7),
      title='Population'
    ),
    Layer(
      'starbucks_stores',
      size_continuous_style('rev_pop', stroke_color='white'),
      default_widget=True,
      title='Revenue per person ($)'
    )
])

result_map

Cool! Now that you have a small dashboard to play with, let's publish it on CARTO so you are able to share it with anyone. To do this, you just need to call the publish method from the Map class:

In [20]:
result_map.publish('starbucks_analysis', password=None)

{'id': '99763034-3e1b-46c4-9e5a-eba0586d97c8',
 'url': 'https://team.carto.com/u/elena-carto/kuviz/99763034-3e1b-46c4-9e5a-eba0586d97c8',
 'name': 'starbucks_analysis',
 'privacy': 'link'}

### Conclusion

Congratulations! You have finished this guide and have a sense about how CARTOframes can speed up your workflow. To continue learning, you can check out a specific [Guide](/developers/cartoframes/guides), the [Reference](/developers/cartoframes/reference) to know everything about a class or a method or check the [Examples](/developers/cartoframes/examples) to see CARTOframes in action.